# Objective - Save & Restore Models 
## Agenda
1. Why do we need to ?
2. Checkpointing
3. 

### 1. Why to save & restore models ?
* Model progress can be saved during—and after—training. 
* This means a model can resume where it left off and avoid long training times. 
* Saving also means you can share your model and others can recreate your work. When publishing research models and techniques, most machine learning practitioners share:
  - code to create the model, and
  - the trained weights, or parameters, for the model
* Sharing this data helps others understand how the model works and try it themselves with new data

In [8]:
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [9]:
NUM_WORDS = 10000

(train_data, train_labels), (test_data, test_labels) = keras.datasets.imdb.load_data(num_words=NUM_WORDS)

def multi_hot_sequences(sequences, labels, dimension):
    for seq,label in zip(sequences,labels):
        result = np.zeros((1, dimension))
        res_label = np.array([label])
        for word_indices in seq:
            result[0,word_indices] = 1.0  
        yield result,res_label

In [23]:
def make_model():
    model = keras.Sequential([
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')])
    
    model.compile(optimizer='adam',
                       loss='binary_crossentropy',
                       metrics=['accuracy', 'binary_crossentropy'])
    
    return model

In [24]:
baseline_model = make_model()

train = multi_hot_sequences(train_data, train_labels, dimension=NUM_WORDS)
test = multi_hot_sequences(test_data, test_labels, dimension=NUM_WORDS)

baseline_history = baseline_model.fit_generator(train,epochs=20,steps_per_epoch=512,verbose=2, validation_data=test, validation_steps= 1000)

Epoch 1/20
512/512 - 2s - loss: 0.6245 - accuracy: 0.5872 - binary_crossentropy: 0.6730 - val_loss: 0.4792 - val_accuracy: 0.7780 - val_binary_crossentropy: 0.4792
Epoch 2/20
512/512 - 1s - loss: 0.4876 - accuracy: 0.7825 - binary_crossentropy: 0.4679 - val_loss: 0.4397 - val_accuracy: 0.7890 - val_binary_crossentropy: 0.4397
Epoch 3/20
512/512 - 1s - loss: 0.3791 - accuracy: 0.8221 - binary_crossentropy: 0.3699 - val_loss: 0.3486 - val_accuracy: 0.8460 - val_binary_crossentropy: 0.3486
Epoch 4/20
512/512 - 1s - loss: 0.4046 - accuracy: 0.8137 - binary_crossentropy: 0.4354 - val_loss: 0.4130 - val_accuracy: 0.8090 - val_binary_crossentropy: 0.4130
Epoch 5/20
512/512 - 1s - loss: 0.3751 - accuracy: 0.8531 - binary_crossentropy: 0.3775 - val_loss: 0.3510 - val_accuracy: 0.8420 - val_binary_crossentropy: 0.3510
Epoch 6/20
512/512 - 1s - loss: 0.3818 - accuracy: 0.8702 - binary_crossentropy: 0.3764 - val_loss: 0.3334 - val_accuracy: 0.8640 - val_binary_crossentropy: 0.3334
Epoch 7/20
512/5

### Saving checkpoints during training
* You can use a trained model without having to retrain it, or pick-up training where you left off—in case the training process was interrupted.

In [12]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [25]:
train = multi_hot_sequences(train_data, train_labels, dimension=NUM_WORDS)
test = multi_hot_sequences(test_data, test_labels, dimension=NUM_WORDS)

model_chkpt = make_model()

model_chkpt_history = model_chkpt.fit_generator(train,epochs=20,steps_per_epoch=512,verbose=2, validation_data=test, validation_steps= 1000, callbacks=[cp_callback])

Epoch 1/20

Epoch 00001: saving model to training_1/cp.ckpt
512/512 - 2s - loss: 0.5994 - accuracy: 0.5626 - binary_crossentropy: 0.6662 - val_loss: 0.4771 - val_accuracy: 0.7740 - val_binary_crossentropy: 0.4771
Epoch 2/20

Epoch 00002: saving model to training_1/cp.ckpt
512/512 - 1s - loss: 0.4893 - accuracy: 0.7753 - binary_crossentropy: 0.4538 - val_loss: 0.4398 - val_accuracy: 0.7850 - val_binary_crossentropy: 0.4398
Epoch 3/20

Epoch 00003: saving model to training_1/cp.ckpt
512/512 - 1s - loss: 0.3867 - accuracy: 0.8300 - binary_crossentropy: 0.3720 - val_loss: 0.3878 - val_accuracy: 0.8360 - val_binary_crossentropy: 0.3878
Epoch 4/20

Epoch 00004: saving model to training_1/cp.ckpt
512/512 - 1s - loss: 0.4074 - accuracy: 0.8338 - binary_crossentropy: 0.4417 - val_loss: 0.3994 - val_accuracy: 0.8290 - val_binary_crossentropy: 0.3994
Epoch 5/20

Epoch 00005: saving model to training_1/cp.ckpt
512/512 - 1s - loss: 0.3959 - accuracy: 0.8584 - binary_crossentropy: 0.4057 - val_loss:

In [28]:
baseline_model.evaluate_generator(test, steps=1000)

[0.32284281072010856, 0.861, 0.3228429]

* Predicting using untrained model

In [26]:
untrained_model = make_model()

In [27]:
untrained_model.evaluate_generator(test, steps=1000)

[0.6916080796122551, 0.512, 0.69160795]

### Loading saved model
* Load weights from checkpoint to re-evaluate

In [29]:
untrained_model.load_weights(checkpoint_path)

In [30]:
untrained_model.evaluate_generator(test, steps=1000)

[0.33172763868992866, 0.863, 0.33172756]

 * Checkpointing after every 5 epochs

In [33]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

# Create a new model instance
model = make_model()

# Train the model with the new callback
train = multi_hot_sequences(train_data, train_labels, dimension=NUM_WORDS)
test = multi_hot_sequences(test_data, test_labels, dimension=NUM_WORDS)

history = model.fit_generator(train,epochs=20,
                                                steps_per_epoch=512,verbose=2, 
                                                validation_data=test, 
                                                validation_steps= 1000, callbacks=[cp_callback])

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

W0730 15:45:27.568769 139740642637632 callbacks.py:859] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


Epoch 1/20
512/512 - 2s - loss: 0.5965 - accuracy: 0.5883 - binary_crossentropy: 0.6555 - val_loss: 0.4588 - val_accuracy: 0.7880 - val_binary_crossentropy: 0.4588
Epoch 2/20
512/512 - 1s - loss: 0.4839 - accuracy: 0.7742 - binary_crossentropy: 0.4537 - val_loss: 0.4331 - val_accuracy: 0.7980 - val_binary_crossentropy: 0.4331
Epoch 3/20
512/512 - 1s - loss: 0.3737 - accuracy: 0.8377 - binary_crossentropy: 0.3550 - val_loss: 0.3731 - val_accuracy: 0.8370 - val_binary_crossentropy: 0.3731
Epoch 4/20
512/512 - 1s - loss: 0.4144 - accuracy: 0.8235 - binary_crossentropy: 0.4452 - val_loss: 0.4285 - val_accuracy: 0.8160 - val_binary_crossentropy: 0.4285
Epoch 5/20

Epoch 00005: saving model to training_2/cp-0005.ckpt
512/512 - 1s - loss: 0.3908 - accuracy: 0.8550 - binary_crossentropy: 0.3987 - val_loss: 0.3726 - val_accuracy: 0.8300 - val_binary_crossentropy: 0.3726
Epoch 6/20
512/512 - 1s - loss: 0.3877 - accuracy: 0.8530 - binary_crossentropy: 0.3816 - val_loss: 0.3375 - val_accuracy: 0.8

* Checkpoint files contain the learned weights
* Manually saving weights

In [37]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = make_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
model.evaluate_generator(test, steps=1000)

[0.3628490496046576, 0.852, 0.36284903]

### Saving the entire model
* The model and optimizer can be saved to a file that contains both their state (weights and variables) and the model configuration. 
* This allows you to export a model so it can be used without access to the original Python code. Since the optimizer-state is recovered, you can resume training from exactly where you left off.

In [38]:
model.save('my_trained_model.h5')

In [39]:
model = keras.models.load_model('my_trained_model.h5')

W0730 17:44:49.810069 139740642637632 hdf5_format.py:197] Sequential models without an `input_shape` passed to the first layer cannot reload their optimizer state. As a result, your model isstarting with a freshly initialized optimizer.


* Resuming training from where it stopped

In [ ]:
train = multi_hot_sequences(train_data, train_labels, dimension=NUM_WORDS)
test = multi_hot_sequences(test_data, test_labels, dimension=NUM_WORDS)

history = model.fit_generator(train,epochs=20,
                                                steps_per_epoch=512,verbose=2, 
                                                validation_data=test, 
                                                validation_steps= 1000, callbacks=[cp_callback])

Epoch 1/20
512/512 - 1s - loss: 0.1864 - accuracy: 0.9199 - binary_crossentropy: 0.1864 - val_loss: 0.3917 - val_accuracy: 0.8570 - val_binary_crossentropy: 0.3917
Epoch 2/20
512/512 - 1s - loss: 0.1822 - accuracy: 0.9141 - binary_crossentropy: 0.1822 - val_loss: 0.3983 - val_accuracy: 0.8520 - val_binary_crossentropy: 0.3983
Epoch 3/20

Epoch 00003: saving model to training_2/cp-0003.ckpt
512/512 - 1s - loss: 0.1724 - accuracy: 0.9375 - binary_crossentropy: 0.1724 - val_loss: 0.3423 - val_accuracy: 0.8700 - val_binary_crossentropy: 0.3423
Epoch 4/20
512/512 - 1s - loss: 0.1578 - accuracy: 0.9336 - binary_crossentropy: 0.1578 - val_loss: 0.3886 - val_accuracy: 0.8590 - val_binary_crossentropy: 0.3886
Epoch 5/20
512/512 - 1s - loss: 0.1824 - accuracy: 0.9336 - binary_crossentropy: 0.1824 - val_loss: 0.3714 - val_accuracy: 0.8630 - val_binary_crossentropy: 0.3714
Epoch 6/20
512/512 - 1s - loss: 0.1914 - accuracy: 0.9277 - binary_crossentropy: 0.1914 - val_loss: 0.3742 - val_accuracy: 0.8